In [ ]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.webdriver import WebDriver 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By 
from selenium.webdriver.remote.webelement import WebElement 
import time 
import html 
import pandas
option = Options() 
option.add_experimental_option("detach", True) 
driver:WebDriver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
action_chain = ActionChains(driver)


In [8]:
import pandas as pd
pd.read_csv("product data and links.csv")

,brand index,product name,product price,json description list,json image url list,json metadata
0,0,OnePlus Nord CE 3 Lite 5G (Chromatic G...,19999,"["" Camera: 108 MP Main Camera with EIS; 2MP De...","[""https://m.media-amazon.com/images/I/61LB+d0v...","{""Brand"": ""OnePlus"", ""Model Name"": ""OnePlus No..."
1,0,OnePlus Nord CE 3 Lite 5G (Pastel Lime...,19999,"["" Camera: 108 MP Main Camera with EIS; 2MP De...","[""https://m.media-amazon.com/images/I/61QRgOgB...","{""Brand"": ""OnePlus"", ""Model Name"": ""OnePlus No..."
2,0,"OnePlus 10R 5G (Sierra Black, 8GB RAM,...",34999,"["" Camera: 50MP Main Camera with Sony IMX766 a...","[""https://m.media-amazon.com/images/I/71BoiXkr...","{""Brand"": ""OnePlus"", ""Model Name"": ""OnePlus 10..."
3,0,"OnePlus 11R 5G (Sonic Black, 8GB RAM, ...",39999,"["" Camera: Sensor: 50MP Main Camera with Sony ...","[""https://m.media-amazon.com/images/I/71qjUzUt...","{""Brand"": ""OnePlus"", ""Model Name"": ""OnePlus 11..."
4,0,"OnePlus Nord CE 2 Lite 5G (Blue Tide, ...",17999,"["" Camera: 64MP Main Camera with EIS; 2MP Dept...","[""https://m.media-amazon.com/images/I/71AvQd3V...","{""Brand"": ""OnePlus"", ""Model Name"": ""OnePlus No..."
...,...,...,...,...,...,...
2184,51,"Fevicryl Acrylic Colors Lilac Kit, 60m...",100,"["" Bright and transparent "", "" Provides full ...","[""https://m.media-amazon.com/images/I/71XGYjj2...","{""Brand"": ""Fevicryl"", ""Colour"": ""Multicolor"", ..."
2185,51,Pidilite Rangeela Creative Clay Pack D...,35,"["" Enhance your child's ability to recognize d...","[""https://m.media-amazon.com/images/I/614-iWvM...","{""Brand"": ""Pidilite"", ""Colour"": ""Multi"", ""Age ..."
2186,51,Pidilite Fevicol Xpres Adhesive - Bond...,520,"["" Excellent Water resistance it can be used i...","[""https://m.media-amazon.com/images/I/51+jA88s...","{""Brand"": ""Pidilite"", ""Material"": ""Wood"", ""Com..."
2187,51,Pidilite Fevicryl Acrylic Painting Col...,279,"["" VIBRANT SHADES - The paint presents fresh a...","[""https://m.media-amazon.com/images/I/51qBGY7s...","{""Brand"": ""Fevicryl"", ""Colour"": ""Red"", ""Finish..."


In [ ]:
action_chain = ActionChains(driver)


In [ ]:
from selenium.webdriver.common.action_chains import ActionChains


In [ ]:
brand_data = pd.read_csv("brand data and links.csv", index_col = "Index")
def check_for_amazon_signout():
    try:
        driver.find_element(By.XPATH, ".//a/span[text()=\"Sign in\"]")
        input("AMAZON SIGNED YOU OUT SIGN IN !!!!")
        return True
    except:
        return False


In [ ]:
brand_data.iloc[0,:]

In [ ]:
def hower_on_product_image_icons():
    HOWER_TIME = 0.15
    image_icon_eles = driver.find_element(By.ID, "altImages").find_elements(By.XPATH,".//li[contains(@class,'item')]")
    for image_icon_element in image_icon_eles:
        action_chain.move_to_element(image_icon_element).perform()
        time.sleep(HOWER_TIME) 

In [ ]:
def get_product_metadata():
    res = dict()
    try:
        row_elements = driver.find_elements(By.XPATH, ".//div[@id='productOverview_feature_div']/div/table/tbody/tr")
#         print(row_elements)
        for row_ele in row_elements:
            value_elements = row_ele.find_elements(By.XPATH, "./td/span")
            assert len(value_elements) == 2, "Unexpected schema encountered"
            key = value_elements[0].get_attribute("innerHTML")
            value = value_elements[1].get_attribute("innerHTML")
            res[key] = value
        return res
    except Exception as e:
        return dict()

In [ ]:
def get_product_data_from_product_link(product_link):
#     driver.get(product_link)
    if(check_for_amazon_signout()):
        driver.get(product_link)
#     hower_on_product_image_icons()
    product_name  = driver.find_element(By.ID, "productTitle").get_attribute("innerHTML")
    product_price = int(driver.find_element(By.XPATH, \
                    ".//div[@id='corePriceDisplay_desktop_feature_div']//span[contains(@class, 'a-price-whole')]")\
                    .get_attribute("innerHTML").replace(',',''))
    product_image_elements = driver.find_elements(By.XPATH, ".//span[@data-action=\"main-image-click\"]//img[@src]")
    product_image_links = list(map(lambda x: x.get_attribute("src"), product_image_elements))
    product_description_ele_list = driver.find_elements(By.XPATH, ".//div[@id='feature-bullets']/ul/li/span")
    product_description_list = list(map(lambda x: x.get_attribute("innerHTML"), product_description_ele_list))
    product_metadata = get_product_metadata()
    return product_name, product_price, list(product_image_links), product_description_list, product_metadata
get_product_data_from_product_link("https://www.amazon.in/OnePlus-Nord-Chromatic-128GB-Storage/dp/B0BY8MCQ9S/ref=sr_1_1?qid=1687439310&refinements=p_89%3AOnePlus&rnid=3837712031&s=electronics&sr=1-1")

In [ ]:
def get_products_data_list_for_brand(index:int = 0):
    link = brand_data.iloc[0,2]
#     driver.get(link)
    res = []
    if(check_for_amazon_signout()):
        driver.get(link)
    product_list = driver.find_elements(By.XPATH, ".//div[@data-component-type='s-search-result']//a/div/img/../..")
    product_links = map(lambda x: x.get_attribute("href"), product_list)
    return list(product_links)
get_products_data_list_for_brand()[0]